In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
endpoint_url = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(endpoint_url)

In [12]:
# Gets the Drug Label
query = """
SELECT ?label
WHERE {
  wd:Q220426 rdfs:label ?label.
  FILTER (lang(?label) = "en")
}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['results']['bindings'][0]['label']['value'])

zopiclone


In [76]:
# Get All Interacting Drugs wrt. another Drug (e.g., Zopiclone)
drug = "wd:Q220426"
interaction_property = "wdt:P769"
side_effect_property = "wdt:P1909"

query = f"""
SELECT ?interactingDrugLabel 
WHERE {{
  {drug} {interaction_property} ?interactingDrug.
  ?interactingDrug rdfs:label ?interactingDrugLabel.
  FILTER (LANG(?interactingDrugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for interacting_drug in results['results']['bindings']:
    print(interacting_drug['interactingDrugLabel']['value'])

codeine
pethidine
loxapine
morphine
alfentanil
(RS)-methadone
hydromorphone
dihydrocodeine
oxycodone
fentanyl
sodium oxybate
buprenorphine
tapentadol
sufentanil
periciazine
tramadol


##### p:P769 is used to find the statement node describing a significant drug interaction.
##### ps:P769 extracts the drug involved in the interaction (e.g., Pethidine, Periciazine).
##### pq:P1909 extracts the side effects associated with the interaction (e.g., "Depressed level of consciousness").

In [2]:
drug = "wd:Q220426"  # Zopiclone
interaction_property = "p:P769"
interaction_statement = "ps:P769"
side_effect_qualifier = "pq:P1909"

query = f"""
SELECT ?interactingDrugLabel (GROUP_CONCAT(?sideEffectLabel; separator=", ") AS ?sideEffects) WHERE {{
  {drug} {interaction_property} ?statement.
  ?statement {interaction_statement} ?interactingDrug.
  ?interactingDrug rdfs:label ?interactingDrugLabel.
  FILTER (LANG(?interactingDrugLabel) = "en")

  OPTIONAL {{
    ?statement {side_effect_qualifier} ?sideEffect.
    ?sideEffect rdfs:label ?sideEffectLabel.
    FILTER (LANG(?sideEffectLabel) = "en")
  }}
}} GROUP BY ?interactingDrugLabel
"""

# Set query and format
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

# Execute query
results = sparql.query().convert()

# Prepare data for JSON
output_data = {}

for binding in results['results']['bindings']:
    interacting_drug_label = binding['interactingDrugLabel']['value']
    side_effects = binding.get('sideEffects', {}).get('value', 'No side effects listed')
    side_effects_list = side_effects.split(', ') if side_effects != 'No side effects listed' else []
    output_data[interacting_drug_label] = side_effects_list

# Write to JSON file
import json
output_file = "drug_side_effects.json"
with open(output_file, "w") as file:
    json.dump(output_data, file, indent=4)

print(f"Results successfully written to {output_file}")

Results successfully written to drug_side_effects.json


In [5]:
# Get all instances of "medicines"
has_use = "wdt:P366"
medication = "wd:Q12140"

query = f"""
SELECT ?drug ?drugLabel
WHERE {{
  ?drug {has_use} {medication}.  
  ?drug rdfs:label ?drugLabel.
  FILTER (LANG(?drugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
i=1
meds = []
for drug in results['results']['bindings']:
    print(str(i) + ":", drug['drugLabel']['value'])
    meds.append(drug['drugLabel']['value'])
    i += 1
  

import csv
output_file = 'medications.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    for med in meds:
        writer.writerow([med])

print(f"Medications have been written to {output_file}.")

1: carbon dioxide
2: DL-amphetamine
3: sodium bicarbonate
4: gabapentin
5: rac-salbutamol
6: ampicillin
7: Diclofenac
8: ethanol
9: isosorbide dinitrate
10: L-glutamine
11: aspartame
12: arginine vasopressin
13: lercanidipine
14: megestrol
15: neostigmine
16: boceprevir
17: phenobarbital
18: theophylline
19: magnesium hydroxide
20: tramadol
21: L-menthol
22: papaverine
23: phenytoin
24: oxycodone
25: fentanyl
26: cytarabine
27: tryptophan
28: buprenorphine
29: dextromethorphan
30: metformin
31: root vegetable
32: isoflurophate
33: 6-thioguanine
34: desoximetasone
35: ajmaline
36: natalizumab
37: physostigmine
38: butabarbital
39: meptazinol
40: ofatumumab
41: fluoxymesterone
42: sertraline
43: (+)-pseudoephedrine
44: ethoxzolamide
45: mitiglinide
46: cisatracurium besylate
47: phenprocoumon
48: benzylpenicillin
49: cilostazol
50: fusidic acid
51: iohexol
52: ambrisentan
53: heptabarbital
54: phenylacetic acid
55: D-sorbitol
56: fluorescein
57: insulin detemir
58: diphenylpyraline
59: f

In [47]:
# Get the count of all medicines
has_use = "wdt:P366"
medication = "wd:Q12140"

query = f"""
SELECT (COUNT(?drug) AS ?drugCount)
WHERE {{
  ?drug {has_use} {medication}.  
  ?drug rdfs:label ?drugLabel.
  FILTER (LANG(?drugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print("Medicine Count:", results['results']['bindings'][0]['drugCount']['value'])


Medicine Count: 2689


In [55]:
# Find Chemical Substance used as Medicine through Query by name
query = f"""
SELECT ?drug ?drugLabel
WHERE {{
  ?drug rdfs:label "zopiclone"@en.
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['results']['bindings'][0]['drug']['value'])

http://www.wikidata.org/entity/Q220426


In [ ]:
# Get Best Matched Meds based on Wrongly Type Input Medication
from fuzzywuzzy import process

input_name = "zopliklon"  

matches = process.extract(input_name, meds, limit=5)
print(matches)


[('zopiclone', 78), ('eszopiclone', 70), ('xylometazoline', 60), ('tetrahydrozoline', 60), ('zotepine', 59)]
